In [1]:
import pandas as pd

import ipywidgets as widgets
from IPython.display import clear_output, HTML

from wimbledon.vis.Visualise import Visualise

## Select Dates to Show

In [2]:
# date picker to choose a start date
widg_start = widgets.DatePicker(
                    description='Start Date:',
                    value=pd.datetime.now(),
                    disabled=False
                )

# date picker to choose an end date
widg_end = widgets.DatePicker(
                    description='End Date:',
                    value=pd.datetime.now()+pd.Timedelta('365 days'),
                    disabled=False
                )

freq_dict = {'Monthly':'MS','Weekly':'W-MON','Daily':'D'}

# radio buttons to choose the frequency of dates to display in tables/heatmaps
widg_freq = widgets.RadioButtons(
    options=freq_dict.keys(),
    description='Frequency:',
    disabled=False
)

display(widg_start, widg_end, widg_freq)

DatePicker(value=datetime.datetime(2019, 5, 30, 15, 40, 32, 651868), description='Start Date:')

DatePicker(value=datetime.datetime(2020, 5, 29, 15, 40, 32, 656413), description='End Date:')

RadioButtons(description='Frequency:', options=('Monthly', 'Weekly', 'Daily'), value='Monthly')

## Load Data

In [3]:
vis = Visualise(start_date=pd.to_datetime(widg_start.value), 
                end_date=pd.to_datetime(widg_end.value), 
                freq=freq_dict[widg_freq.value],
                init_harvest=False)

Enpoint: whoami | Time: 0.5165243950000002 | Cache: False
Enpoint: people/460554 | Time: 0.4320497089999997 | Cache: False

AUTHENTICATED USER:
Jack Roberts jroberts@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.4358677830000004 | Cache: False
PROJECTS
Enpoint: projects | Time: 0.6993509499999999 | Cache: False
ROLES
Enpoint: roles | Time: 0.48318643599999955 | Cache: False
PEOPLE
Enpoint: people | Time: 0.5686588749999997 | Cache: False
PLACEHOLDERS
Enpoint: placeholders | Time: 0.43876145499999986 | Cache: False
MILESTONES
Enpoint: milestones | Time: 0.43734584200000004 | Cache: False
ASSIGNMENTS
Enpoint: assignments | Time: 0.6871800389999994 | Cache: False
DONE! (4.7s)


In [4]:
def update_vis_params(change):
    vis.START_DATE = pd.to_datetime(widg_start.value)
    vis.END_DATE = pd.to_datetime(widg_end.value)
    vis.FREQ = freq_dict[widg_freq.value]

widg_start.observe(update_vis_params, names='value')
widg_end.observe(update_vis_params, names='value')
widg_freq.observe(update_vis_params, names='value')

## Whiteboard Visualisations

In [5]:
display(HTML(vis.whiteboard('person')))

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
display(HTML(vis.whiteboard('project')))

## Show Someone's Project Assignment

In [ ]:
## dropdown list to select a person
widg_person_select = widgets.Dropdown(options=sorted([vis.fc.get_person_name(person_id) for person_id in vis.fc.people.index]),
                                 description='Person:')

def widg_person_select_alloc(change):
    """Function called by widgets to update the person allocation plot and table.
    The change argument isn't used but it's required for widget.observe."""
    
    # clear the previous output, then display the widgets again
    clear_output()
    display(widg_person_select)
    
    # get the person_id from the selected full name (need to first split into first and last names)
    names = widg_person_select.value
    person_id = vis.fc.get_person_id(names)
    
    # plot the person allocations
    vis.plot_allocations(person_id, 'person')
    
    # print the person allocations table
    #display(vis.table_allocations(person_id, 'person'))
    vis.heatmap_allocations(person_id, 'person')
    
# setup the event listeners
widg_person_select.observe(widg_person_select_alloc, names='value')

# display the widgets
display(widg_person_select)

# show results with initial values
widg_person_select_alloc(None)

## Show Everyone's Total Project Allocation Over Time

In [ ]:
vis.heatmap_allocations('ALL_PEOPLE', 'institute');

## Show Who's Assigned to a Project

In [ ]:
## dropdown list to select a project
widg_project_select = widgets.Dropdown(options=sorted([vis.fc.get_project_name(project_id) for project_id in vis.fc.projects.index]),
                                 description='project:')

def widg_project_select_alloc(change):
    """Function called by widgets to update the project allocation plot and table.
    The change argument isn't used but it's required for widget.observe."""
    
    # clear the previous output, then display the widgets again
    clear_output()
    display(widg_project_select)
    
    # get the project_id from the selected full name (need to first split into first and last names)
    project_id = vis.fc.get_project_id(widg_project_select.value)
    
    # plot the project allocations
    vis.plot_allocations(project_id, 'institute')
    
    # plot a heatmap of the allocations
    vis.heatmap_allocations(project_id, 'institute')

# setup the event listeners
widg_project_select.observe(widg_project_select_alloc, names='value')

# display the widgets
display(widg_project_select)

# show results with initial values
widg_project_select_alloc(None)

## Show the Total Resources Required by All Projects

In [ ]:
vis.heatmap_allocations('PROJECT_REQUIREMENTS','institute');

## Show the Total Resources Allocated to Each Project

In [ ]:
vis.heatmap_allocations('PROJECT_TOTALS', 'institute');

## Show the Additional Resources Needed to be Allocated to Each Project

In [ ]:
vis.heatmap_allocations('PROJECT_NETALLOC', 'institute');

## Compare the Capcity of the Team to the Requirements of the Projects

In [ ]:
vis.plot_capacity_check();

## Show Allocations for a Placeholder

In [ ]:
## dropdown list to select a placeholder
widg_place_select = widgets.Dropdown(options=sorted([vis.fc.get_placeholder_name(idx) for idx in vis.fc.placeholders.index]),
                                 description='Placeholder:')

def widg_select_alloc(change):
    """Function called by widgets to update the allocation plot and table.
    The change argument isn't used but it's required for widget.observe."""
    
    # clear the previous output, then display the widgets again
    clear_output()
    display(widg_place_select)
    
    # get the person_id from the selected full name (need to first split into first and last names)
    placeholder_id = vis.fc.get_placeholder_id(widg_place_select.value)
    
    # plot the person allocations
    vis.plot_allocations(placeholder_id, 'placeholder')
    
    # print the person allocations table
    #display(vis.table_allocations(person_id, 'person'))
    vis.heatmap_allocations(placeholder_id, 'placeholder')
    
# setup the event listeners
widg_place_select.observe(widg_select_alloc, names='value')

# display the widgets
display(widg_place_select)

# show results with initial values
widg_select_alloc(None)

In [ ]:
vis.fc.project_reqs